In [ ]:
import os
import cv2
from google.colab import drive
import shutil

drive.mount('/content/drive', force_remount=True)

original_dataset_path = '/content/drive/MyDrive/accident-dataset'

yolo_dataset_path = '/content/drive/MyDrive/yolo_accident_dataset'
os.makedirs(yolo_dataset_path, exist_ok=True)

class_names = ["Non Accident", "Accident"]
class_to_id = {name: idx for idx, name in enumerate(class_names)}

def convert_to_yolo(split):
    src_img_dir = os.path.join(original_dataset_path, split)
    src_subdirs = [os.path.join(src_img_dir, cls) for cls in class_names]

    dst_img_dir = os.path.join(yolo_dataset_path, 'images', split)
    dst_label_dir = os.path.join(yolo_dataset_path, 'labels', split)
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_label_dir, exist_ok=True)

    for cls, subdir in zip(class_names, src_subdirs):
        if not os.path.exists(subdir):
            print(f"Warning: {subdir} does not exist.")
            continue
        for img_file in os.listdir(subdir):
            if not img_file.endswith(('.jpg', '.jpeg', '.png')):
                continue

            img_path = os.path.join(subdir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Error loading {img_path}")
                continue
            h, w = img.shape[:2]

            center_x, center_y = w / 2, h / 2
            box_w, box_h = w * 0.8, h * 0.8
            norm_center_x = center_x / w
            norm_center_y = center_y / h
            norm_w = box_w / w
            norm_h = box_h / h

            annotation = f"{class_to_id[cls]} {norm_center_x:.6f} {norm_center_y:.6f} {norm_w:.6f} {norm_h:.6f}"

            dst_img_path = os.path.join(dst_img_dir, img_file)
            shutil.copy(img_path, dst_img_path)

            label_file = os.path.splitext(img_file)[0] + '.txt'
            dst_label_path = os.path.join(dst_label_dir, label_file)
            with open(dst_label_path, 'w') as f:
                f.write(annotation)

for split in ['train', 'val', 'test']:
    convert_to_yolo(split)

yaml_content = f"""path: {yolo_dataset_path}
train: images/train
val: images/val
test: images/test
nc: 2
names: {class_names}
"""
yaml_path = os.path.join(yolo_dataset_path, 'data.yaml')
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

if os.path.exists(yaml_path):
    print("data.yaml created successfully at:", yaml_path)
else:
    print("Error: data.yaml not created.")

Mounted at /content/drive
data.yaml created successfully at: /content/drive/MyDrive/yolo_accident_dataset/data.yaml


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import os
from google.colab import drive
import torch
from ultralytics import YOLO

drive.mount('/content/drive', force_remount=True)

dataset_path = '/content/drive/MyDrive/yolo_accident_dataset'
yaml_path = os.path.join(dataset_path, 'data.yaml')

if not os.path.exists(yaml_path):
    print(f"Error: {yaml_path} does not exist. Run the conversion script first.")
    exit()

model = YOLO('yolov8m.pt')

device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Training on: {device if device == 'cpu' else torch.cuda.get_device_name(device)}")

training_params = {
    'data': yaml_path,           # Dataset config
    'epochs': 100,               # More epochs for better learning
    'imgsz': 640,                # Larger image size for detail
    'batch': 16,                 # Adjust based on GPU memory
    'patience': 20,              # Early stopping
    'lr0': 0.001,                # Lower learning rate
    'optimizer': 'AdamW',        # Better generalization
    'cos_lr': True,              # Cosine LR scheduler
    'augment': True,             # Enable augmentation
    'mosaic': 1.0,               # Mosaic augmentation
    'mixup': 0.2,                # Mixup augmentation
    'hsv_h': 0.015,              # Hue augmentation
    'hsv_s': 0.7,                # Saturation augmentation
    'hsv_v': 0.4,                # Value augmentation
    'degrees': 10.0,             # Rotation
    'translate': 0.1,            # Translation
    'scale': 0.5,                # Scaling
    'shear': 2.0,                # Shear
    'perspective': 0.0001,       # Perspective
    'flipud': 0.5,               # Vertical flip
    'fliplr': 0.5,               # Horizontal flip
    'project': '/content/drive/MyDrive/yolo_training/runs',
    'name': 'accident_detection',
    'exist_ok': True,
    'device': device,            # Use detected device
    'cache': True,               # Cache images
}

# Train the model
results = model.train(**training_params)

# Evaluate on validation set
metrics = model.val()

# Print metrics
print(f"Validation mAP@0.5: {metrics.box.map50:.4f}")
print(f"Validation mAP@0.5:0.95: {metrics.box.map:.4f}")

# Export the model
model.export(format='onnx')
print(f"Training complete. Results saved to: {training_params['project']}/{training_params['name']}")
print(f"Exported model: {training_params['project']}/{training_params['name']}/weights/best.onnx")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


100%|██████████| 49.7M/49.7M [00:01<00:00, 46.3MB/s]


Training on: Tesla T4
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/yolo_accident_dataset/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=/content/drive/MyDrive/yolo_training/runs, name=accident_detection, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

100%|██████████| 755k/755k [00:00<00:00, 87.3MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...

                   all         98         98      0.868      0.878      0.955      0.949
          Non Accident         52         52      0.915      0.865      0.965      0.957
              Accident         46         46      0.821      0.891      0.945      0.941
Speed: 0.3ms preprocess, 27.3ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_training/runs/accident_detection
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 45.6±12.3 MB/s, size: 231.3 KB)


val: Scanning /content/drive/MyDrive/yolo_accident_dataset/labels/val.cache... 98 images, 0 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.1GB RAM): 100%|██████████| 98/98 [00:01<00:00, 89.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  1.75it/s]


                   all         98         98      0.915      0.866      0.957      0.948
          Non Accident         52         52       0.93      0.885      0.967      0.954
              Accident         46         46      0.899      0.848      0.947      0.942
Speed: 1.6ms preprocess, 31.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_training/runs/accident_detection
Validation mAP@0.5: 0.9570
Validation mAP@0.5:0.95: 0.9480
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from '/content/drive/MyDrive/yolo_training/runs/accident_detection/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 293.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14

In [ ]:
import cv2
import os
from google.colab import drive
from ultralytics import YOLO

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the trained YOLOv8 model
model_path = '/content/drive/MyDrive/yolo_training/runs/accident_detection/weights/best.pt'
model = YOLO(model_path)
print("Model loaded successfully from:", model_path)

# Input and output video paths
input_video_path = '/content/drive/MyDrive/data/videoplayback (online-video-cutter.com).mp4'  # Your test video
output_video_path = '/content/accident_detection_output5.mp4'  # Output video path

# Open the input video
video = cv2.VideoCapture(input_video_path)

# Check if video opened successfully
if not video.isOpened():
    print(f"Error: Could not open video at {input_video_path}")
    exit()

# Get video properties
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Check if VideoWriter initialized
if not out.isOpened():
    print("Error: Could not initialize VideoWriter")
    video.release()
    exit()

# Process the video frame by frame
frame_count = 0
while True:
    ret, frame = video.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Perform inference with YOLOv8
    results = model.predict(frame, conf=0.2, iou=0.7, verbose=False)  # Confidence threshold 0.5, IoU 0.7

    # Annotate the frame with detections
    for result in results:
        boxes = result.boxes  # Bounding box objects
        for box in boxes:
            # Extract coordinates, confidence, and class
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = box.conf[0]  # Confidence score
            cls = int(box.cls[0])  # Class index
            label = f"{model.names[cls]} {conf:.2f}"  # e.g., "Accident 0.95"

            # Draw bounding box and label
            color = (0, 255, 0) if model.names[cls] == "Non Accident" else (0, 0, 255)  # Green for Non Accident, Red for Accident
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Write the annotated frame to the output video
    out.write(frame)
    frame_count += 1

    # Optional: Print progress
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames")

# Release resources
video.release()
out.release()
cv2.destroyAllWindows()
print(f"Processed {frame_count} frames. Output video saved to: {output_video_path}")

# Verify output file
if os.path.exists(output_video_path) and os.path.getsize(output_video_path) > 0:
    print("Output video created successfully.")
else:
    print("Warning: Output video is empty or not created.")

Mounted at /content/drive
Model loaded successfully from: /content/drive/MyDrive/yolo_training/runs/accident_detection/weights/best.pt
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Processed 500 frames
Processed 600 frames
Processed 700 frames
Processed 800 frames
Processed 900 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1700 frames
Processed 1800 frames
Processed 1900 frames
Processed 2000 frames
Processed 2100 frames
Processed 2200 frames
End of video or error reading frame.
Processed 2298 frames. Output video saved to: /content/accident_detection_output5.mp4
Output video created successfully.
